
# Bottleneck With Expansion Residual Block (ResNet Bottleneck) — En İnceden En Üste

Bu notlar “**Bottleneck + Expansion**” residual bloğun **neden ortaya çıktığını**, **ne problemi çözdüğünü**, **Basic residual** ve **Pre-activation (ResNet v2)** ile **farklarını** en dipten en tepeye kadar anlatır.

> Hedef: Bir CNN/Detection backbone’u tasarlarken “**neden bottleneck?**”, “**expansion ne işe yarıyor?**”, “**pre mi post mu?**”, “**projection ne zaman?**” sorularını refleks haline getirmek.



---

## 0) Büyük Resim: Neyi çözmeye çalışıyoruz?

Derin ağlar (çok katman) teoride daha güçlüdür ama pratikte iki büyük sıkıntı vardır:

1. **Optimizasyon zorluğu**: Derinleşince eğitim zorlaşır (loss düşmez / çok yavaş düşer).
2. **Gradient akışı**: Çok katmanda gradient sinyali zayıflar veya dengesizleşir.

Residual bağlantı (skip) fikri:
- Ağın öğrenmesi gerekeni “tam fonksiyon” yerine **düzeltme (residual)** haline getirir.

\[
y = x + F(x)
\]

Böylece **kimlik (identity)** yol her zaman açıktır; gradient için otoyol oluşur.



---

## 1) Basic Residual Block (ResNet-18/34) — Referans Noktası

### Yapı (Post-activation / ResNet v1)
Klasik basic block (çoğunlukla):

- Main path: `Conv3×3 → BN → ReLU → Conv3×3 → BN`
- Skip path: `Identity` (boyut tutuyorsa)
- Toplama: `y = x + F(x)`
- Son: `ReLU(y)`  (v1’de genellikle burada)

Kısaca:

```bash
x ────────────────┐
      Conv-BN-ReLU│
      Conv-BN     │
                  ├─ add ─ ReLU ─→ y
F(x) ─────────────┘
```

### Güçlü yön:
- Derinleşmeyi pratikte mümkün kılar.

### Zayıf yön (derinlik çok artınca):
- ReLU/BN sıralaması nedeniyle **skip yol “tam identity” gibi davranmakta zorlanabilir**.
- Çok derin ağlarda optimizasyon hâlâ zorlaşabilir.



---

## 2) Pre-Activation Residual Block (ResNet v2) — “Sıra”yı düzeltme

Pre-act’in temel farkı: **BN+Act, konvdan önce** gelir.

### Pre-act basic block (ResNet v2)
- Main path: `BN → ReLU → Conv → BN → ReLU → Conv`
- Skip path: mümkünse **pure identity**
- Toplama: `y = x + F(BN/ReLU(x))`
- Genellikle **add sonrası ReLU yok** (ya da blok sonunda değil, bir sonraki bloğun başında var)

Kısaca:

```bash
x ────────────────┐
 BN-ReLU-Conv      │
 BN-ReLU-Conv      │
                   ├─ add ─→ y
F(·) ──────────────┘
```

### Neden işe yarıyor?
- Skip yol daha “temiz identity” kalır.
- Gradient daha rahat akar.
- Çok derin ağlarda stabilite artar.

> Özet: Basic residual “residual fikrini” getirir, pre-act ise “bu fikri çok derinde daha stabil yapan sıralama”yı getirir.



---

## 3) Bottleneck With Expansion — “Niye çıktık buraya?”

Şimdi asıl mesele: **ağ derinleşti, yetmedi; ayrıca genişlemek istiyoruz**.

### Problem
Derinliği artırmak kolaylaştı ama:
- **Her bloğa 3×3 conv eklemek pahalı** (özellikle kanal sayısı büyüdükçe).
- “Daha güçlü temsil” için kanalı büyütmek istersen FLOPs uçar.

### Çözüm fikri
3×3 konv **en pahalı** kısımdır. Peki 3×3 konvu **daha düşük/daha kontrollü kanalda** çalıştırıp,
kanal karışımını **1×1 konvlarla** halledersek?

Bu düşünce: **Bottleneck**.

### Bottleneck (ResNet-50/101/152) yapısı
Klasik ResNet bottleneck:

- `1×1 Conv` (**reduce / inner channels**)
- `3×3 Conv` (asıl uzamsal iş)
- `1×1 Conv` (**expand / output channels**)

Kısaca:

```bash
x ────────────────┐
 1×1 (reduce)      │
 3×3 (spatial)     │
 1×1 (expand)      │
                   ├─ add ─→ y
F(x) ──────────────┘
```

### Expansion nedir?
ResNet bottleneck’te genelde:
- Bloğun “iç kanalı” = `C_mid`
- Bloğun çıkışı = `C_mid * expansion`  (çoğunlukla **4**)

Yani son 1×1 ile **kanalı büyütür**, böylece bloklar arası temsil gücü artar.

> Not: Bu “ResNet bottleneck expansion”dır. MobileNetV2’deki “inverted bottleneck” başka mantık (expand→depthwise→project).



---

## 4) “Neden 1×1 – 3×3 – 1×1?” (en ince sebepler)

### 4.1 1×1 konv ne yapar?
- Kanallar arası karışım (mixing).
- Hesap ucuzdur (uzamsal kernel yok).

### 4.2 3×3 konv ne yapar?
- Uzamsal desen öğrenimi (kenar, köşe, doku, şekil).
- Hesap pahalıdır.

### 4.3 Bottleneck stratejisi
- 3×3’ü **daha küçük iç kanalda** çalıştır (ucuzlat).
- 1×1 ile kanalı önce “hazırla”, sonra “genişlet”.

Böylece:
- Parametre/FLOPs kontrol altında
- Temsil gücü yüksek



---

## 5) Basic vs Bottleneck vs Pre-act — net farklar

### 5.1 Basic Residual Block
- 2 adet 3×3 conv
- Daha basit
- Küçük/orta derinlikte iyi (ResNet-18/34)

### 5.2 Bottleneck With Expansion (ResNet v1)
- 1×1 reduce + 3×3 + 1×1 expand
- 3×3 sayısı 1’e düşer (maliyet düşer)
- Çok derin/çok geniş ağlar için ideal (ResNet-50+)

### 5.3 Pre-activation Bottleneck (ResNet v2)
- Bottleneck yapısı aynı, ama **BN+Act konvdan önce**
- Skip identity daha temiz
- Çok derinlerde daha stabil



---

## 6) Projection (Downsample) ne zaman gerekiyor?

Eğer `x` ile `F(x)` aynı boyutta değilse toplama yapılamaz.

Bu iki durumda olur:
1. **Stride=2** ile uzamsal boyut küçülür (H,W yarıya iner).
2. Kanal sayısı değişir (C_in ≠ C_out).

Çözüm: skip path’e `1×1 conv (projection)` koymak:

\[
y = W_s x + F(x)
\]



---

## 7) Parametre/FLOPs sezgisi (neden bottleneck daha mantıklı?)

Aynı giriş/çıkış kanalını düşünelim: `C_out = 256`

- Basic block’ta iki adet 3×3 konv 256 kanalda çalışırsa pahalı.
- Bottleneck’te iç kanal `C_mid = 64` seçip 3×3’ü 64 kanalda çalıştırırsın.

Sonuç: Aynı bütçede çok daha derin model mümkün.



---

## 8) Kod: Basic v1, Bottleneck v1, Bottleneck v2 (Pre-act)

Aşağıdaki kodlar, blok mantığını net görmek için minimal tutuldu.


In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

def conv1x1(in_ch, out_ch, stride=1, bias=False):
    return nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, padding=0, bias=bias)

def conv3x3(in_ch, out_ch, stride=1, bias=False):
    return nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=bias)


class BasicBlockV1(nn.Module):
    # ResNet v1 style (post-act): Conv-BN-ReLU, Conv-BN, add, ReLU
    expansion = 1
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_ch, out_ch, stride)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.conv2 = conv3x3(out_ch, out_ch, 1)
        self.bn2   = nn.BatchNorm2d(out_ch)

        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Sequential(
                conv1x1(in_ch, out_ch, stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        identity = x if self.proj is None else self.proj(x)

        y = F.relu(self.bn1(self.conv1(x)), inplace=True)
        y = self.bn2(self.conv2(y))
        y = F.relu(identity + y, inplace=True)
        return y


class BottleneckV1(nn.Module):
    # ResNet v1 bottleneck: 1x1 -> 3x3 -> 1x1(expand), add, ReLU
    expansion = 4
    def __init__(self, in_ch, mid_ch, stride=1):
        super().__init__()
        out_ch = mid_ch * self.expansion

        self.conv1 = conv1x1(in_ch, mid_ch, 1)
        self.bn1   = nn.BatchNorm2d(mid_ch)

        self.conv2 = conv3x3(mid_ch, mid_ch, stride)
        self.bn2   = nn.BatchNorm2d(mid_ch)

        self.conv3 = conv1x1(mid_ch, out_ch, 1)
        self.bn3   = nn.BatchNorm2d(out_ch)

        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Sequential(
                conv1x1(in_ch, out_ch, stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        identity = x if self.proj is None else self.proj(x)

        y = F.relu(self.bn1(self.conv1(x)), inplace=True)
        y = F.relu(self.bn2(self.conv2(y)), inplace=True)
        y = self.bn3(self.conv3(y))

        y = F.relu(identity + y, inplace=True)
        return y


class BottleneckV2PreAct(nn.Module):
    # ResNet v2 bottleneck: BN-ReLU-Conv order, add (often no ReLU here)
    expansion = 4
    def __init__(self, in_ch, mid_ch, stride=1):
        super().__init__()
        out_ch = mid_ch * self.expansion

        self.bn1 = nn.BatchNorm2d(in_ch)
        self.conv1 = conv1x1(in_ch, mid_ch, 1)

        self.bn2 = nn.BatchNorm2d(mid_ch)
        self.conv2 = conv3x3(mid_ch, mid_ch, stride)

        self.bn3 = nn.BatchNorm2d(mid_ch)
        self.conv3 = conv1x1(mid_ch, out_ch, 1)

        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = conv1x1(in_ch, out_ch, stride)

    def forward(self, x):
        x_p = F.relu(self.bn1(x), inplace=True)

        identity = x if self.proj is None else self.proj(x)

        y = self.conv1(x_p)
        y = self.conv2(F.relu(self.bn2(y), inplace=True))
        y = self.conv3(F.relu(self.bn3(y), inplace=True))

        return identity + y



---

## 9) Hızlı doğrulama: Shape ve downsample

- Basic: çıkış kanal = out_ch  
- Bottleneck: çıkış kanal = mid_ch * 4  
- Stride=2: H,W yarıya iner


In [2]:

device = "cuda" if torch.cuda.is_available() else "cpu"

x = torch.randn(2, 64, 32, 32, device=device)

b1 = BasicBlockV1(64, 64, stride=1).to(device)
y1 = b1(x)
print("Basic v1:", tuple(x.shape), "->", tuple(y1.shape))

b2 = BasicBlockV1(64, 128, stride=2).to(device)
y2 = b2(x)
print("Basic v1 (ds):", tuple(x.shape), "->", tuple(y2.shape))

bn1 = BottleneckV1(64, 64, stride=1).to(device)   # out = 64*4=256
y3 = bn1(x)
print("Bottleneck v1:", tuple(x.shape), "->", tuple(y3.shape))

bn2 = BottleneckV1(64, 64, stride=2).to(device)
y4 = bn2(x)
print("Bottleneck v1 (ds):", tuple(x.shape), "->", tuple(y4.shape))

v2 = BottleneckV2PreAct(64, 64, stride=1).to(device)
y5 = v2(x)
print("Bottleneck v2 pre-act:", tuple(x.shape), "->", tuple(y5.shape))


Basic v1: (2, 64, 32, 32) -> (2, 64, 32, 32)
Basic v1 (ds): (2, 64, 32, 32) -> (2, 128, 16, 16)
Bottleneck v1: (2, 64, 32, 32) -> (2, 256, 32, 32)
Bottleneck v1 (ds): (2, 64, 32, 32) -> (2, 256, 16, 16)
Bottleneck v2 pre-act: (2, 64, 32, 32) -> (2, 256, 32, 32)



---

## 10) Detection için pratik kararlar

- **ResNet-18/34 tarzı**: basic block
- **ResNet-50+ backbone**: bottleneck
- **Çok derin + stabilite**: pre-act bottleneck (v2)
- **Boyut değişiyorsa**: projection şart
- **Attention ekliyorsan**: çoğunlukla `F(x)` yoluna ekle, skip’i temiz bırak



---

## 11) 1 paragraf final özet

Bottleneck with expansion residual block, basic residual bloğun “derin ve geniş ağları aynı hesap bütçesinde çalıştırma” ihtiyacından doğmuş halidir. İki pahalı 3×3 yerine 3×3’ü tek yere indirir, kanalı 1×1 ile önce düzenler sonra expansion ile büyütür. Böylece hem temsil gücü artar hem FLOPs/parametre kontrol altında kalır. Pre-activation varyantı (ResNet v2) ise aynı tasarımı BN+Activation’ı konvlardan önce alarak daha temiz identity skip ve daha stabil gradient akışı sağlar.
